In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [59]:
# Pyspark unable to infer the schema so I had to do it myself
custom_schema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", TimestampType(), True),
    StructField("Booking ID", StringType(), True),
    StructField("Booking Status", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Vehicle Type", StringType(), True),
    StructField("Pickup Location", StringType(), True),
    StructField("Drop Location", StringType(), True),
    StructField("Avg VTAT", DoubleType(), True),
    StructField("Avg CTAT", DoubleType(), True),
    StructField("Cancelled Rides by Customer", IntegerType(), True),
    StructField("Reason for cancelling by Customer", StringType(), True),
    StructField("Cancelled Rides by Driver", IntegerType(), True),
    StructField("Driver Cancellation Reason", StringType(), True),
    StructField("Incomplete Rides", IntegerType(), True),
    StructField("Incomplete Rides Reason", StringType(), True),
    StructField("Booking Value", DoubleType(), True),
    StructField("Ride Distance", DoubleType(), True),
    StructField("Driver Ratings", DoubleType(), True),
    StructField("Customer Rating", DoubleType(), True),
    StructField("Payment Method", StringType(), True)
])

In [60]:
spark = SparkSession.builder.appName("UberAnalysis").config("spark.sql.adaptive.enabled", "true").getOrCreate()

df = spark.read.csv("static/ncr_ride_bookings.csv", header=True, schema=custom_schema)
df.show(5)
(df.count(), len(df.columns))

+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|      Date|               Time|      Booking ID| Booking Status|     Customer ID| Vehicle Type|    Pickup Location|    Drop Location|Avg VTAT|Avg CTAT|Cancelled Rides by Customer|Reason for cancelling by Customer|Cancelled Rides by Driver|Driver Cancellation Reason|Incomplete Rides|Incomplete Rides Reason|Booking Value|Ride Distance|Driver Ratings|Customer Rating|Payment Method|
+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+--------------------

(150000, 21)

In [61]:
booking_analysis = df.groupBy("Booking Status") \
    .agg(count("*").alias("count")) \
    .withColumn("percentage", col("count") / df.count() * 100) \
    .orderBy(desc("count"))
booking_analysis.show()

+--------------------+-----+-----------------+
|      Booking Status|count|       percentage|
+--------------------+-----+-----------------+
|           Completed|93000|             62.0|
| Cancelled by Driver|27000|             18.0|
|     No Driver Found|10500|7.000000000000001|
|Cancelled by Cust...|10500|7.000000000000001|
|          Incomplete| 9000|              6.0|
+--------------------+-----+-----------------+



In [62]:
revenue_by_payment = df.filter(col("Booking Status") == "Completed").groupBy("Payment Method") \
        .agg(sum("Booking Value").alias("Total Revenue")).orderBy(desc("Total Revenue"))

revenue_by_payment.show(truncate=False)

+--------------+-------------+
|Payment Method|Total Revenue|
+--------------+-------------+
|UPI           |2.1274591E7  |
|Cash          |1.1756071E7  |
|Uber Wallet   |5669070.0    |
|Credit Card   |4753767.0    |
|Debit Card    |3807075.0    |
+--------------+-------------+



In [68]:
# Turn all null val to 0
df_zero_filled = df.na.fill(0)
df_zero_filled.show(5)

+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|      Date|               Time|      Booking ID| Booking Status|     Customer ID| Vehicle Type|    Pickup Location|    Drop Location|Avg VTAT|Avg CTAT|Cancelled Rides by Customer|Reason for cancelling by Customer|Cancelled Rides by Driver|Driver Cancellation Reason|Incomplete Rides|Incomplete Rides Reason|Booking Value|Ride Distance|Driver Ratings|Customer Rating|Payment Method|
+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+--------------------

In [ ]:
vehicle_performance = df.groupBy("Vehicle Type").agg(
        count("*").alias("Total Bookings"),
        avg("Ride Distance").alias("Average Ride Distance"),
        avg("Driver Ratings").alias("Average Driver Rating"),
        avg("Customer Rating").alias("Average Customer Rating")
    ).orderBy(desc("Total Bookings"))

vehicle_performance.show()

+-------------+--------------+---------------------+---------------------+-----------------------+
| Vehicle Type|Total Bookings|Average Ride Distance|Average Driver Rating|Average Customer Rating|
+-------------+--------------+---------------------+---------------------+-----------------------+
|         Auto|         37419|    24.61599567184734|     4.23236881882964|      4.401999568127833|
|      Go Mini|         29806|   24.612089962679214|    4.227694215321595|      4.404296727586389|
|     Go Sedan|         27141|    24.60932470793755|    4.231812185176316|      4.409996402014868|
|         Bike|         22517|   24.649546283036038|    4.230055579307396|     4.4039404303833525|
|Premier Sedan|         18111|     24.5989557450264|    4.234864912904378|      4.403457163170992|
|        eBike|         10557|    24.99041498398552|   4.2256144100137325|      4.403953594871011|
|      Uber XL|          4449|   24.402893267651898|    4.238339920948616|      4.404850880344952|
+---------